In [74]:
import datetime
import glob
import os

import cftime as cf
import numpy as np
import xarray as xr
import rioxarray as rxr

In [75]:
dirpath = "/mnt/poseidon/remotesensing/arctic/data/rasters/model_results_tiled_test_06-19-2025"
sdate = datetime.datetime(2019, 6, 1)
edate = datetime.datetime(2019, 9, 30)
filepath = f"{dirpath}/bryophyte_30M-5P-IQR3_mosaic_clipped_masked.tif"

In [76]:
# function to extract PFT name from filename
def extract_pft(fp):
    base = os.path.splitext(os.path.basename(fp))[0]
    parts = base.split('_')
    p = []
    for seg in parts:
        if seg and seg[0].isdigit():
            break
        p.append(seg)
    return '_'.join(p)

In [77]:
# set parameters
dirpath = "/mnt/poseidon/remotesensing/arctic/data/rasters/model_results_tiled_test_06-19-2025"
sdate = datetime.datetime(2019, 6, 1)
edate = datetime.datetime(2019, 9, 30)
chunk_size = 2048

variable_name = extract_pft(filepath)
nc_file = f"{dirpath}/{variable_name}_pavc-raster_arctic-alaska_summer-2019_v1-1.nc"

In [78]:
# start by loading the water mask
mask = rxr.open_rasterio(f"{dirpath}/water_mask_clipped.tif", band_as_variable=True)
mask = mask.rename({"y": "lat", "x": "lon", "band_1": "water_mask"})
del mask["spatial_ref"]
for attr in ("scale_factor", "add_offset", "_FillValue"):
    del mask["water_mask"].attrs[attr]
del mask.attrs["AREA_OR_POINT"]

In [79]:
# clear and set mask attributes
mask["water_mask"].encoding.clear()
mask["water_mask"].attrs.update({
    "long_name": "Water Mask",
    "standard_name": "status_flag",
    "valid_range": [np.byte(0), np.byte(1)],
    "flag_values": [np.byte(0), np.byte(1)],
    "flag_meanings": "not_water water",
})

In [80]:
mask

<xarray.Dataset> Size: 13GB
Dimensions:     (lon: 178421, lat: 71368)
Coordinates:
  * lon         (lon) float64 1MB -173.1 -173.1 -173.1 ... -141.0 -141.0 -141.0
  * lat         (lat) float64 571kB 71.37 71.37 71.37 ... 58.55 58.55 58.55
Data variables:
    water_mask  (lat, lon) uint8 13GB ...

In [81]:
ds = rxr.open_rasterio(filepath, band_as_variable=True)
ds = ds.chunk({"y": chunk_size, "x": chunk_size})
ds = ds.rename({
    "x":"lon", 
    "y": "lat",
    "band_1":"cover"
})

In [82]:
# add long_name to cover and remove some attrs
ds["cover"].attrs["long_name"] = f"percent total cover of {variable_name}s"
for attr in ("scale_factor", "add_offset", "_FillValue"):
    del ds["cover"].attrs[attr]
del ds.attrs["AREA_OR_POINT"]
del ds["spatial_ref"]

In [83]:
# set cover encoding
cover_encoding = {
    'dtype': 'float32',
    'zlib': True,
    'complevel': 6,
    'shuffle': True,
    '_FillValue': np.float32(-9999),
    'chunksizes': (1, chunk_size, chunk_size)
}

In [84]:
tb_arr = np.asarray(
    [
        [cf.DatetimeNoLeap(sdate.year, sdate.month, sdate.day)],
        [cf.DatetimeNoLeap(edate.year, edate.month, edate.day)],
    ]
).T
tb_da = xr.DataArray(tb_arr, dims=("time", "nv"))
time_mid = tb_da.mean(dim="nv")

In [85]:
ds = ds.expand_dims(time=time_mid)
ds["time_bounds"] = tb_da

In [86]:
ds["time"].attrs = {"axis": "T", "long_name": "time"}
ds["time"].encoding["units"] = f"days since {sdate.strftime('%Y-%m-%d %H:%M:%S')}"
ds["time"].encoding["calendar"] = "noleap"
ds["time"].encoding["bounds"] = "time_bounds"

In [87]:
ds["water_mask"] = mask["water_mask"]

In [88]:
ds = ds.reindex(lat=list(reversed(ds.lat)))
ds["lat"].attrs = {"axis": "Y", "long_name": "latitude", "units": "degrees_north"}
ds["lon"].attrs = {"axis": "X", "long_name": "longitude", "units": "degrees_east"}

In [89]:
# set _FillValue to None for time, lat, lon
for var in list(ds.coords) + [v for v in ds.data_vars if v != "cover"]:
    ds[var].encoding["_FillValue"] = None

In [90]:
# add global attributes
ds.attrs = {
    "title": f"Pan-Arctic Vegetation Cover (PAVC) Gridded: {variable_name.replace('_',' ')}",
    "version": "v1.1",
    "institution": "Oak Ridge National Laboratory",
    "source": ("Total fractional cover estimated by combining 20m Sentinel and ArcticDEM-derived "
                "predictors with high-quality plot samples based on fine-tuned random forest regression models"),
    "references": "",
    "comment": "The water mask identifies pixels where ndwi > 0 and ndvi <0.3",
    "Conventions": "CF-1.12",
}

In [91]:
encoding = {
    'cover': cover_encoding,
    'time': ds['time'].encoding,
    'lat': ds['lat'].encoding,
    'lon': ds['lon'].encoding,
    'time_bounds': ds['time_bounds'].encoding,
    'water_mask': dict(dtype="byte", _FillValue=np.int8(-1))
}

In [92]:
ds

<xarray.Dataset> Size: 64GB
Dimensions:      (time: 1, lon: 178421, lat: 71368, nv: 2)
Coordinates:
  * time         (time) object 8B 2019-07-31 12:00:00
  * lon          (lon) float64 1MB -173.1 -173.1 -173.1 ... -141.0 -141.0 -141.0
  * lat          (lat) float64 571kB 58.55 58.55 58.55 ... 71.37 71.37 71.37
Dimensions without coordinates: nv
Data variables:
    cover        (time, lat, lon) float32 51GB dask.array<chunksize=(1, 2039, 2048), meta=np.ndarray>
    time_bounds  (time, nv) object 16B 2019-06-01 00:00:00 2019-09-30 00:00:00
    water_mask   (lat, lon) uint8 13GB ...
Attributes:
    title:        Pan-Arctic Vegetation Cover (PAVC) Gridded: bryophyte
    version:      v1.1
    institution:  Oak Ridge National Laboratory
    source:       Total fractional cover estimated by combining 20m Sentinel ...
    references:   
    comment:      The water mask identifies pixels where ndwi > 0 and ndvi <0.3
    Conventions:  CF-1.12